In [15]:
import threading
import time
import random
from queue import Queue, Empty, Full

1. Задача про обідаючих філософів. П'ять філософів відпочивають у пансіонаті. Кожен із них 
може або вдаватися до роздумів, або обідати, відвідавши їдальню. У їдальні знаходиться 
круглий стіл, за яким є місця для кожного з філософів, п'ять тарілок, п'ять виделок, 
розташованих між тарілками, та страва спагетті у центрі столу. Кожен філософ для того щоб 
пообідати бере одночасно дві вилки і приступає до трапези, по завершенню якої він кладе 
вилки на стіл і уходить, повертаючись до роздумів.

In [16]:
class DiningPhilosophers1:
    def __init__(self, n=5):
        self.n = n
        self.forks = [threading.Lock() for _ in range(n)]
        self.eating_lock = threading.Lock()
        
    def philosopher(self, phil_id):
        left_fork = phil_id
        right_fork = (phil_id + 1) % self.n
        
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.5))
            
            # Одночасне взяття обох вилок
            with self.eating_lock:
                self.forks[left_fork].acquire()
                self.forks[right_fork].acquire()
                
                print(f"Філософ {phil_id} їсть")
                time.sleep(random.uniform(0.1, 0.3))
                
                self.forks[left_fork].release()
                self.forks[right_fork].release()
                print(f"Філософ {phil_id} закінчив їсти")
    
    def run(self):
        threads = []
        for i in range(self.n):
            t = threading.Thread(target=self.philosopher, args=(i,))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()

# Запуск
dp1 = DiningPhilosophers1()
dp1.run()

Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 1 їсть
Філософ 1 закінчив їсти
Філософ 1 роздумує...
Філософ 2 їсть
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 3 їсть
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 0 їсть
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 4 їсть
Філософ 4 закінчив їсти
Філософ 4 роздумує...
Філософ 1 їсть
Філософ 1 закінчив їсти
Філософ 1 роздумує...
Філософ 2 їсть
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 3 їсть
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 0 їсть
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 4 їсть
Філософ 4 закінчив їсти
Філософ 4 роздумує...
Філософ 1 їсть
Філософ 1 закінчив їсти
Філософ 2 їсть
Філософ 2 закінчив їсти
Філософ 0 їсть
Філософ 0 закінчив їсти
Філософ 3 їсть
Філософ 3 закінчив їсти
Філософ 4 їсть
Філософ 4 закінчив їсти


2. Перетворення задачі 1 про обідаючих філософів для випадку, коли ліву та праву вилки 
беруть по черзі (не одночасно).

In [17]:
class DiningPhilosophers2:
    def __init__(self, n=5):
        self.n = n
        self.forks = [threading.Lock() for _ in range(n)]
        
    def philosopher(self, phil_id):
        left_fork = phil_id
        right_fork = (phil_id + 1) % self.n
        
        # Уникнення deadlock: завжди беремо вилки в певному порядку
        first_fork = min(left_fork, right_fork)
        second_fork = max(left_fork, right_fork)
        
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.5))
            
            self.forks[first_fork].acquire()
            print(f"Філософ {phil_id} взяв вилку {first_fork}")
            
            self.forks[second_fork].acquire()
            print(f"Філософ {phil_id} взяв вилку {second_fork}")
            
            print(f"Філософ {phil_id} їсть")
            time.sleep(random.uniform(0.1, 0.3))
            
            self.forks[second_fork].release()
            self.forks[first_fork].release()
            print(f"Філософ {phil_id} закінчив їсти")
    
    def run(self):
        threads = []
        for i in range(self.n):
            t = threading.Thread(target=self.philosopher, args=(i,))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()

# Запуск
dp2 = DiningPhilosophers2()
dp2.run()

Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 2 взяв вилку 2
Філософ 2 взяв вилку 3
Філософ 2 їсть
Філософ 0 взяв вилку 0
Філософ 0 взяв вилку 1
Філософ 0 їсть
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 4 взяв вилку 0
Філософ 4 взяв вилку 4
Філософ 4 їсть
Філософ 3 взяв вилку 3
Філософ 1 взяв вилку 1
Філософ 1 взяв вилку 2
Філософ 1 їсть
Філософ 4 закінчив їстиФілософ 3 взяв вилку 4
Філософ 3 їсть

Філософ 4 роздумує...
Філософ 0 взяв вилку 0
Філософ 1 закінчив їсти
Філософ 1 роздумує...
Філософ 0 взяв вилку 1
Філософ 0 їсть
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 2 взяв вилку 2
Філософ 2 взяв вилку 3
Філософ 2 їсть
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 4 взяв вилку 0
Філософ 4 взяв вилку 4
Філософ 4 їсть
Філософ 2 закінчив їстиФілософ 3 взяв вилку 3

Філософ 2 роздумує...
Філософ 1 взяв вилку 1
Філософ 1 взяв вилку 2
Філософ 1 ї

3. Перетворення задачі 2 для випадку, коли за стіл запрошують одного з філософів, який не 
перешкоджає загальному поглинанню їжі обідаючими.

In [18]:
class DiningPhilosophers3:
    def __init__(self, n=5):
        self.n = n
        self.forks = [threading.Lock() for _ in range(n)]
        self.table_semaphore = threading.Semaphore(n - 1)  # Максимум n-1 за столом
        
    def philosopher(self, phil_id):
        left_fork = phil_id
        right_fork = (phil_id + 1) % self.n
        
        first_fork = min(left_fork, right_fork)
        second_fork = max(left_fork, right_fork)
        
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.5))
            
            self.table_semaphore.acquire()
            print(f"Філософ {phil_id} сів за стіл")
            
            self.forks[first_fork].acquire()
            self.forks[second_fork].acquire()
            
            print(f"Філософ {phil_id} їсть")
            time.sleep(random.uniform(0.1, 0.3))
            
            self.forks[second_fork].release()
            self.forks[first_fork].release()
            
            self.table_semaphore.release()
            print(f"Філософ {phil_id} покинув стіл")
    
    def run(self):
        threads = []
        for i in range(self.n):
            t = threading.Thread(target=self.philosopher, args=(i,))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()

# Запуск
dp3 = DiningPhilosophers3()
dp3.run()



Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 4 сів за стіл
Філософ 4 їсть
Філософ 3 сів за стіл
Філософ 2 сів за стіл
Філософ 0 сів за стіл
Філософ 4 покинув стіл
Філософ 4 роздумує...
Філософ 3 їсть
Філософ 0 їсть
Філософ 1 сів за стіл
Філософ 3 покинув стіл
Філософ 3 роздумує...
Філософ 2 їсть
Філософ 4 сів за стіл
Філософ 0 покинув стіл
Філософ 0 роздумує...
Філософ 4 їсть
Філософ 2 покинув стілФілософ 1 їсть

Філософ 2 роздумує...
Філософ 4 покинув стіл
Філософ 4 роздумує...
Філософ 3 сів за стіл
Філософ 3 їсть
Філософ 2 сів за стіл
Філософ 1 покинув стіл
Філософ 1 роздумує...
Філософ 0 сів за стіл
Філософ 0 їсть
Філософ 4 сів за стіл
Філософ 3 покинув стіл
Філософ 3 роздумує...
Філософ 2 їсть
Філософ 0 покинув стіл
Філософ 0 роздумує...
Філософ 4 їсть
Філософ 1 сів за стіл
Філософ 2 покинув стіл
Філософ 2 роздумує...
Філософ 3 сів за стіл
Філософ 1 їсть
Філософ 4 покинув стіл
Філософ 3 їсть
Філософ 0 сів за с

4. Перетворення задачі 3 для випадку, коли запрошення до столу необов'язкові для розгляду 
філософом.

In [19]:
class DiningPhilosophers4:
    def __init__(self, n=5):
        self.n = n
        self.forks = [threading.Lock() for _ in range(n)]
        self.table_semaphore = threading.Semaphore(n - 1)
        
    def philosopher(self, phil_id):
        left_fork = phil_id
        right_fork = (phil_id + 1) % self.n
        
        first_fork = min(left_fork, right_fork)
        second_fork = max(left_fork, right_fork)
        
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.5))
            
            # Спроба отримати доступ до столу (необов'язкова)
            if self.table_semaphore.acquire(blocking=False):
                print(f"Філософ {phil_id} отримав запрошення і сів за стіл")
                
                self.forks[first_fork].acquire()
                self.forks[second_fork].acquire()
                
                print(f"Філософ {phil_id} їсть")
                time.sleep(random.uniform(0.1, 0.3))
                
                self.forks[second_fork].release()
                self.forks[first_fork].release()
                
                self.table_semaphore.release()
                print(f"Філософ {phil_id} покинув стіл")
            else:
                print(f"Філософ {phil_id} не отримав запрошення, продовжує роздумувати")
    
    def run(self):
        threads = []
        for i in range(self.n):
            t = threading.Thread(target=self.philosopher, args=(i,))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()

# Запуск
dp4 = DiningPhilosophers4()
dp4.run()



Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 3 отримав запрошення і сів за стіл
Філософ 3 їсть
Філософ 0 отримав запрошення і сів за стіл
Філософ 0 їсть
Філософ 1 отримав запрошення і сів за стіл
Філософ 4 отримав запрошення і сів за стіл
Філософ 2 не отримав запрошення, продовжує роздумувати
Філософ 2 роздумує...
Філософ 3 покинув стіл
Філософ 3 роздумує...
Філософ 0 покинув стілФілософ 1 їсть
Філософ 4 їсть

Філософ 0 роздумує...
Філософ 4 покинув стіл
Філософ 4 роздумує...
Філософ 2 отримав запрошення і сів за стіл
Філософ 1 покинув стілФілософ 2 їсть

Філософ 1 роздумує...
Філософ 3 отримав запрошення і сів за стіл
Філософ 0 отримав запрошення і сів за стіл
Філософ 0 їсть
Філософ 0 покинув стіл
Філософ 0 роздумує...
Філософ 4 отримав запрошення і сів за стіл
Філософ 4 їсть
Філософ 2 покинув стіл
Філософ 2 роздумує...
Філософ 1 отримав запрошення і сів за стіл
Філософ 1 їсть
Філософ 4 покинув стіл
Філософ 4 роз

5. Побудувати модель взаємодії процесів у комп'ютері, що має два процесори. магнітний диск 
та принтер.


In [20]:
class ComputerSystem:
    def __init__(self):
        self.cpu_semaphore = threading.Semaphore(2)  # 2 процесори
        self.disk_lock = threading.Lock()
        self.printer_lock = threading.Lock()
        
    def process(self, process_id, needs_disk=True, needs_printer=False):
        print(f"Процес {process_id} чекає на процесор...")
        
        with self.cpu_semaphore:
            print(f"Процес {process_id} виконується на процесорі")
            time.sleep(random.uniform(0.1, 0.3))
            
            if needs_disk:
                with self.disk_lock:
                    print(f"Процес {process_id} використовує диск")
                    time.sleep(random.uniform(0.1, 0.2))
            
            if needs_printer:
                with self.printer_lock:
                    print(f"Процес {process_id} використовує принтер")
                    time.sleep(random.uniform(0.1, 0.2))
            
            print(f"Процес {process_id} завершено")
    
    def run(self, n_processes=6):
        threads = []
        for i in range(n_processes):
            needs_disk = random.choice([True, False])
            needs_printer = random.choice([True, False])
            t = threading.Thread(target=self.process, args=(i, needs_disk, needs_printer))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()

# Запуск
cs = ComputerSystem()
cs.run()



Процес 0 чекає на процесор...
Процес 0 виконується на процесорі
Процес 1 чекає на процесор...
Процес 1 виконується на процесорі
Процес 2 чекає на процесор...
Процес 3 чекає на процесор...
Процес 4 чекає на процесор...
Процес 5 чекає на процесор...
Процес 0 використовує диск
Процес 0 використовує принтерПроцес 1 використовує диск

Процес 0 завершено
Процес 2 виконується на процесорі
Процес 1 завершено
Процес 3 виконується на процесорі
Процес 3 використовує диск
Процес 3 завершено
Процес 4 виконується на процесорі
Процес 2 використовує диск
Процес 2 завершено
Процес 5 виконується на процесорі
Процес 4 використовує диск
Процес 4 завершено
Процес 5 використовує диск
Процес 5 завершено


6. Побудувати модель взаємодії трьох процесів, з яких один пише повідомлення в буфер, інші 
два обробляють повідомлення і поміщають результат у вихідний буфер

In [21]:
class ProducerConsumerTask6:
    def __init__(self):
        self.input_buffer = Queue()   # необмежений
        self.output_buffer = Queue()  # необмежений
        self.running = True

    # ----- WRITER -----
    def writer(self):
        for i in range(10):
            msg = f"Message_{i}"
            self.input_buffer.put(msg)
            print(f"Writer: написав '{msg}' (вхідний буфер розмір: {self.input_buffer.qsize()})")
            time.sleep(random.uniform(0.05, 0.15))

        self.running = False
        print("Writer: завершив запис")

    # ----- PROCESSORS -----
    def processor(self, proc_id):
        while self.running or not self.input_buffer.empty():
            try:
                msg = self.input_buffer.get(timeout=0.1)
            except Empty:
                continue

            result = f"{msg}_processed_by_{proc_id}"
            print(f"Processor {proc_id}: обробив '{msg}'")
            self.output_buffer.put(result)

            time.sleep(random.uniform(0.05, 0.15))

        print(f"Processor {proc_id}: завершив роботу")

    # ----- RUN -----
    def run(self):
        writer_thread = threading.Thread(target=self.writer)
        proc1_thread = threading.Thread(target=self.processor, args=(1,))
        proc2_thread = threading.Thread(target=self.processor, args=(2,))

        writer_thread.start()
        proc1_thread.start()
        proc2_thread.start()

        writer_thread.join()
        proc1_thread.join()
        proc2_thread.join()

        print("\n=== Всі процеси завершені ===")
        print(f"Вихідний буфер містить {self.output_buffer.qsize()} результатів:")
        while not self.output_buffer.empty():
            print("  ", self.output_buffer.get())


# Запуск
pc6 = ProducerConsumerTask6()
pc6.run()


Writer: написав 'Message_0' (вхідний буфер розмір: 1)Processor 1: обробив 'Message_0'

Writer: написав 'Message_1' (вхідний буфер розмір: 1)
Processor 2: обробив 'Message_1'
Writer: написав 'Message_2' (вхідний буфер розмір: 1)
Processor 1: обробив 'Message_2'
Writer: написав 'Message_3' (вхідний буфер розмір: 1)Processor 2: обробив 'Message_3'

Writer: написав 'Message_4' (вхідний буфер розмір: 1)
Processor 2: обробив 'Message_4'
Writer: написав 'Message_5' (вхідний буфер розмір: 1)
Processor 1: обробив 'Message_5'
Writer: написав 'Message_6' (вхідний буфер розмір: 1)Processor 2: обробив 'Message_6'

Writer: написав 'Message_7' (вхідний буфер розмір: 1)
Processor 1: обробив 'Message_7'
Writer: написав 'Message_8' (вхідний буфер розмір: 1)
Processor 2: обробив 'Message_8'
Writer: написав 'Message_9' (вхідний буфер розмір: 1)Processor 1: обробив 'Message_9'

Writer: завершив запис
Processor 2: завершив роботу
Processor 1: завершив роботу

=== Всі процеси завершені ===
Вихідний буфер міс



7. Перетворити модель завдання 6 при обмеженнях на розмір буферів.

In [22]:
class BoundedProducerConsumer:
    def __init__(self, buffer_size=5):
        self.input_buffer = Queue(maxsize=buffer_size)
        self.output_buffer = Queue(maxsize=buffer_size)

        self.running_writer = True
        self.running_processors = True

    def writer(self):
        for i in range(10):
            msg = f"Message_{i}"
            self.input_buffer.put(msg)  # блокує, якщо буфер повний
            print(f"Writer: написав '{msg}' "
                  f"(вхідний буфер: {self.input_buffer.qsize()}/{self.input_buffer.maxsize})")
            time.sleep(random.uniform(0.05, 0.15))

        self.running_writer = False
        print("Writer: завершив роботу")

    def processor(self, proc_id):
        while self.running_writer or not self.input_buffer.empty():
            try:
                msg = self.input_buffer.get(timeout=0.1)
            except Empty:
                continue

            result = f"{msg}_processed_by_{proc_id}"
            print(f"Processor {proc_id}: обробив '{msg}'")

            placed = False
            while not placed and (self.running_writer or not self.input_buffer.empty()):
                try:
                    self.output_buffer.put(result, timeout=0.1)
                    placed = True
                except Full:
                    continue

            time.sleep(random.uniform(0.05, 0.15))

        print(f"Processor {proc_id}: завершив роботу")

    def output_reader(self):
        """Знімає результати з вихідного буфера, щоб процесори не зависали."""
        while self.running_processors or not self.output_buffer.empty():
            try:
                result = self.output_buffer.get(timeout=0.1)
                print(f"OutputReader: отримав '{result}' "
                      f"(вихідний буфер: {self.output_buffer.qsize()}/{self.output_buffer.maxsize})")
            except Empty:
                continue

        print("OutputReader: завершив роботу")

    def run(self):
        writer_thread = threading.Thread(target=self.writer)

        proc1_thread = threading.Thread(target=self.processor, args=(1,))
        proc2_thread = threading.Thread(target=self.processor, args=(2,))

        output_thread = threading.Thread(target=self.output_reader)

        # Start all threads
        writer_thread.start()
        proc1_thread.start()
        proc2_thread.start()
        output_thread.start()

        # Wait writer
        writer_thread.join()
        # Wait processors
        proc1_thread.join()
        proc2_thread.join()

        # сигналізуємо рідеру, що процесори завершено
        self.running_processors = False

        output_thread.join()

        print("\n=== Усі потоки завершили роботу ===")

bpc = BoundedProducerConsumer(buffer_size=3)
bpc.run()


Writer: написав 'Message_0' (вхідний буфер: 1/3)
Processor 1: обробив 'Message_0'
OutputReader: отримав 'Message_0_processed_by_1' (вихідний буфер: 0/3)
Writer: написав 'Message_1' (вхідний буфер: 1/3)
Processor 2: обробив 'Message_1'
OutputReader: отримав 'Message_1_processed_by_2' (вихідний буфер: 0/3)
Writer: написав 'Message_2' (вхідний буфер: 1/3)
Processor 1: обробив 'Message_2'
OutputReader: отримав 'Message_2_processed_by_1' (вихідний буфер: 0/3)
Writer: написав 'Message_3' (вхідний буфер: 1/3)Processor 2: обробив 'Message_3'

OutputReader: отримав 'Message_3_processed_by_2' (вихідний буфер: 0/3)
Writer: написав 'Message_4' (вхідний буфер: 1/3)
Processor 1: обробив 'Message_4'
OutputReader: отримав 'Message_4_processed_by_1' (вихідний буфер: 0/3)
Writer: написав 'Message_5' (вхідний буфер: 1/3)
Processor 2: обробив 'Message_5'
OutputReader: отримав 'Message_5_processed_by_2' (вихідний буфер: 0/3)
Writer: написав 'Message_6' (вхідний буфер: 1/3)Processor 1: обробив 'Message_6'



8. Представити модель задачі 7 для довільного заданого числа процесів, що записують і 
зчитують повідомлення.

In [23]:
class MultiProcessBoundedPC:
    def __init__(self, buffer_size=5, writers_count=1, processors_count=2, messages_per_writer=10):
        self.input_buffer = Queue(maxsize=buffer_size)
        self.output_buffer = Queue(maxsize=buffer_size)

        self.writers_count = writers_count
        self.processors_count = processors_count
        self.messages_per_writer = messages_per_writer

        self.active_writers = writers_count
        self.active_processors = processors_count

        self.output_count = 0               
        self.output_count_lock = threading.Lock()

    def writer(self, writer_id):
        for i in range(self.messages_per_writer):
            msg = f"W{writer_id}_msg_{i}"

            self.input_buffer.put(msg)
            print(f"Writer {writer_id}: написав '{msg}' "
                  f"(вхідний буфер {self.input_buffer.qsize()}/{self.input_buffer.maxsize})")

            time.sleep(random.uniform(0.05, 0.15))

        self.active_writers -= 1
        print(f"Writer {writer_id}: завершив роботу")

    def processor(self, proc_id):
        while self.active_writers > 0 or not self.input_buffer.empty():
            try:
                msg = self.input_buffer.get(timeout=0.1)
            except Empty:
                continue

            result = f"{msg}_processed_by_{proc_id}"
            print(f"Processor {proc_id}: обробив '{msg}'")

            placed = False
            while not placed and (self.active_writers > 0 or not self.input_buffer.empty()):
                try:
                    self.output_buffer.put(result, timeout=0.1)
                    placed = True
                except Full:
                    continue

            time.sleep(random.uniform(0.05, 0.15))

        self.active_processors -= 1
        print(f"Processor {proc_id}: завершив роботу")

    def output_reader(self):
        while self.active_processors > 0 or not self.output_buffer.empty():
            try:
                res = self.output_buffer.get(timeout=0.1)
            except Empty:
                continue

            # ЛОГІЧНИЙ ПІДРАХУНОК РЕЗУЛЬТАТІВ
            with self.output_count_lock:
                self.output_count += 1

            print(f"OutputReader: отримав '{res}' "
                  f"(вихідний буфер {self.output_buffer.qsize()}/{self.output_buffer.maxsize})")

        print("OutputReader: завершив роботу")

    def run(self):
        threads = []

        # Writers
        for i in range(self.writers_count):
            t = threading.Thread(target=self.writer, args=(i,))
            threads.append(t)
            t.start()

        # Processors
        for p in range(self.processors_count):
            t = threading.Thread(target=self.processor, args=(p,))
            threads.append(t)
            t.start()

        # Output reader (1)
        reader_thread = threading.Thread(target=self.output_reader)
        reader_thread.start()

        # Wait all writers and processors
        for t in threads:
            t.join()

        # Wait reader
        reader_thread.join()

        # ПРАВИЛЬНИЙ РЕЗУЛЬТАТ
        print("\n=== Всі потоки завершені ===")
        print(f"Загальна кількість результатів: {self.output_count}")


model = MultiProcessBoundedPC(
    buffer_size=3,
    writers_count=3,           # Кількість writer-процесів
    processors_count=4,        # Кількість processor-процесів
    messages_per_writer=5      # Повідомлень від кожного writer
)
model.run()


Writer 0: написав 'W0_msg_0' (вхідний буфер 1/3)
Writer 1: написав 'W1_msg_0' (вхідний буфер 2/3)
Writer 2: написав 'W2_msg_0' (вхідний буфер 3/3)
Processor 0: обробив 'W0_msg_0'
Processor 1: обробив 'W1_msg_0'
Processor 2: обробив 'W2_msg_0'
OutputReader: отримав 'W0_msg_0_processed_by_0' (вихідний буфер 2/3)
OutputReader: отримав 'W1_msg_0_processed_by_1' (вихідний буфер 1/3)
OutputReader: отримав 'W2_msg_0_processed_by_2' (вихідний буфер 0/3)
Writer 1: написав 'W1_msg_1' (вхідний буфер 1/3)
Processor 3: обробив 'W1_msg_1'
OutputReader: отримав 'W1_msg_1_processed_by_3' (вихідний буфер 0/3)
Writer 2: написав 'W2_msg_1' (вхідний буфер 1/3)
Processor 2: обробив 'W2_msg_1'
OutputReader: отримав 'W2_msg_1_processed_by_2' (вихідний буфер 0/3)
Writer 0: написав 'W0_msg_1' (вхідний буфер 1/3)
Processor 0: обробив 'W0_msg_1'
OutputReader: отримав 'W0_msg_1_processed_by_0' (вихідний буфер 0/3)
Writer 0: написав 'W0_msg_2' (вхідний буфер 1/3)Processor 2: обробив 'W0_msg_2'

OutputReader: отрим

9. Побудувати модель ліфта що перевозить людей для чотириповерхового будинку. Кнопки 
виклику знаходяться на кожному поверсі.

In [24]:
class Elevator:
    def __init__(self, n_floors=4):
        self.n_floors = n_floors
        self.current_floor = 0
        self.requests = []
        self.lock = threading.Lock()
        self.running = True
        
    def call_elevator(self, person_id, from_floor, to_floor):
        with self.lock:
            self.requests.append((person_id, from_floor, to_floor))
            print(f"Особа {person_id}: викликала ліфт з {from_floor} на {to_floor} поверх")
    
    def operate(self):
        while self.running or self.requests:
            with self.lock:
                if not self.requests:
                    time.sleep(0.1)
                    continue
                
                person_id, from_floor, to_floor = self.requests.pop(0)
            
            # Їдемо до пасажира
            while self.current_floor != from_floor:
                direction = 1 if from_floor > self.current_floor else -1
                self.current_floor += direction
                print(f"Ліфт: поверх {self.current_floor}")
                time.sleep(0.2)
            
            print(f"Ліфт: підібрав особу {person_id} на поверсі {from_floor}")
            
            # Їдемо до призначення
            while self.current_floor != to_floor:
                direction = 1 if to_floor > self.current_floor else -1
                self.current_floor += direction
                print(f"Ліфт: поверх {self.current_floor}")
                time.sleep(0.2)
            
            print(f"Ліфт: висадив особу {person_id} на поверсі {to_floor}")
    
    def run(self):
        elevator_thread = threading.Thread(target=self.operate)
        elevator_thread.start()
        
        # Симуляція викликів
        people = [
            (0, 0, 3),
            (1, 2, 0),
            (2, 1, 3),
            (3, 3, 1)
        ]
        
        for person_id, from_floor, to_floor in people:
            time.sleep(random.uniform(0.3, 0.7))
            self.call_elevator(person_id, from_floor, to_floor)
        
        time.sleep(1)
        self.running = False
        elevator_thread.join()

# Запуск
elev = Elevator()
elev.run()



Особа 0: викликала ліфт з 0 на 3 поверх
Ліфт: підібрав особу 0 на поверсі 0
Ліфт: поверх 1
Ліфт: поверх 2
Ліфт: поверх 3
Особа 1: викликала ліфт з 2 на 0 поверх
Ліфт: висадив особу 0 на поверсі 3
Ліфт: поверх 2
Ліфт: підібрав особу 1 на поверсі 2
Ліфт: поверх 1
Особа 2: викликала ліфт з 1 на 3 поверх
Ліфт: поверх 0
Ліфт: висадив особу 1 на поверсі 0
Ліфт: поверх 1
Особа 3: викликала ліфт з 3 на 1 поверх
Ліфт: підібрав особу 2 на поверсі 1
Ліфт: поверх 2
Ліфт: поверх 3
Ліфт: висадив особу 2 на поверсі 3
Ліфт: підібрав особу 3 на поверсі 3
Ліфт: поверх 2
Ліфт: поверх 1
Ліфт: висадив особу 3 на поверсі 1


10. Побудувати модель задачі 9 з обмеженнями на кількість пасажирів ліфту.

In [25]:
class LimitedElevator:
    def __init__(self, n_floors=4, max_capacity=2):
        self.n_floors = n_floors
        self.max_capacity = max_capacity
        self.current_floor = 0
        self.requests = []
        self.passengers = []
        self.lock = threading.Lock()
        self.running = True
        
    def call_elevator(self, person_id, from_floor, to_floor):
        with self.lock:
            self.requests.append((person_id, from_floor, to_floor))
            print(f"Особа {person_id}: викликала ліфт з {from_floor} на {to_floor} поверх")
    
    def operate(self):
        while self.running or self.requests or self.passengers:
            with self.lock:
                if not self.requests and not self.passengers:
                    time.sleep(0.1)
                    continue
                
                # Висадка пасажирів
                remaining = []
                for person_id, to_floor in self.passengers:
                    if to_floor == self.current_floor:
                        print(f"Ліфт: висадив особу {person_id} на поверсі {to_floor}")
                    else:
                        remaining.append((person_id, to_floor))
                self.passengers = remaining
                
                # Підбір нових пасажирів
                remaining_requests = []
                for person_id, from_floor, to_floor in self.requests:
                    if from_floor == self.current_floor and len(self.passengers) < self.max_capacity:
                        self.passengers.append((person_id, to_floor))
                        print(f"Ліфт: підібрав особу {person_id} (заповненість: {len(self.passengers)}/{self.max_capacity})")
                    else:
                        remaining_requests.append((person_id, from_floor, to_floor))
                self.requests = remaining_requests
                
                # Визначення наступного поверху
                next_floor = self.current_floor
                if self.passengers:
                    next_floor = self.passengers[0][1]
                elif self.requests:
                    next_floor = self.requests[0][1]
            
            if next_floor != self.current_floor:
                direction = 1 if next_floor > self.current_floor else -1
                self.current_floor += direction
                print(f"Ліфт: поверх {self.current_floor}")
                time.sleep(0.2)
    
    def run(self):
        elevator_thread = threading.Thread(target=self.operate)
        elevator_thread.start()
        
        people = [
            (0, 0, 3),
            (1, 0, 2),
            (2, 0, 3),
            (3, 2, 0)
        ]
        
        for person_id, from_floor, to_floor in people:
            time.sleep(random.uniform(0.2, 0.5))
            self.call_elevator(person_id, from_floor, to_floor)
        
        time.sleep(2)
        self.running = False
        elevator_thread.join()

# Запуск
lim_elev = LimitedElevator(max_capacity=2)
lim_elev.run()



Особа 0: викликала ліфт з 0 на 3 поверх
Ліфт: підібрав особу 0 (заповненість: 1/2)
Ліфт: поверх 1
Ліфт: поверх 2
Особа 1: викликала ліфт з 0 на 2 поверх
Ліфт: поверх 3
Особа 2: викликала ліфт з 0 на 3 поверх
Ліфт: висадив особу 0 на поверсі 3
Ліфт: поверх 2
Ліфт: поверх 1
Особа 3: викликала ліфт з 2 на 0 поверх
Ліфт: поверх 0
Ліфт: підібрав особу 1 (заповненість: 1/2)
Ліфт: підібрав особу 2 (заповненість: 2/2)
Ліфт: поверх 1
Ліфт: поверх 2
Ліфт: висадив особу 1 на поверсі 2
Ліфт: підібрав особу 3 (заповненість: 2/2)
Ліфт: поверх 3
Ліфт: висадив особу 2 на поверсі 3
Ліфт: поверх 2
Ліфт: поверх 1
Ліфт: поверх 0
Ліфт: висадив особу 3 на поверсі 0


11. Побудувати модель складання іспиту групою із чотирьох студентів одному екзаменатору.

In [26]:
class Exam:
    def __init__(self):
        self.examiner_lock = threading.Lock()
        
    def student(self, student_id):
        print(f"Студент {student_id}: готується до іспиту")
        time.sleep(random.uniform(0.1, 0.3))
        
        print(f"Студент {student_id}: чекає на екзаменатора")
        with self.examiner_lock:
            print(f"Студент {student_id}: складає іспит")
            time.sleep(random.uniform(0.3, 0.5))
            grade = random.randint(60, 100)
            print(f"Студент {student_id}: отримав оцінку {grade}")
    
    def run(self, n_students=4):
        threads = []
        for i in range(n_students):
            t = threading.Thread(target=self.student, args=(i,))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()

# Запуск
exam = Exam()
exam.run()



Студент 0: готується до іспиту
Студент 1: готується до іспиту
Студент 2: готується до іспиту
Студент 3: готується до іспиту
Студент 2: чекає на екзаменатора
Студент 2: складає іспит
Студент 3: чекає на екзаменатора
Студент 0: чекає на екзаменатора
Студент 1: чекає на екзаменатора
Студент 2: отримав оцінку 100
Студент 3: складає іспит
Студент 3: отримав оцінку 68
Студент 0: складає іспит
Студент 0: отримав оцінку 68
Студент 1: складає іспит
Студент 1: отримав оцінку 72


12. Побудувати модель перукарні, в якій працює три перукарі та є п'ять сидячих місць у 
кімнаті для очікування клієнтів.

In [27]:
class Barbershop:
    def __init__(self, n_barbers=3, n_seats=5):
        self.barbers = threading.Semaphore(n_barbers)
        self.seats = threading.Semaphore(n_seats)
        
    def client(self, client_id):
        print(f"Клієнт {client_id}: прийшов до перукарні")
        
        if self.seats.acquire(blocking=False):
            print(f"Клієнт {client_id}: сів у кімнаті очікування")
            
            self.barbers.acquire()
            print(f"Клієнт {client_id}: стрижеться")
            self.seats.release()
            
            time.sleep(random.uniform(0.3, 0.6))
            
            print(f"Клієнт {client_id}: підстрижений")
            self.barbers.release()
        else:
            print(f"Клієнт {client_id}: немає місць, йде")
    
    def run(self, n_clients=10):
        threads = []
        for i in range(n_clients):
            t = threading.Thread(target=self.client, args=(i,))
            threads.append(t)
            t.start()
            time.sleep(random.uniform(0.05, 0.15))
        
        for t in threads:
            t.join()

# Запуск
barber = Barbershop()
barber.run()



Клієнт 0: прийшов до перукарні
Клієнт 0: сів у кімнаті очікування
Клієнт 0: стрижеться
Клієнт 1: прийшов до перукарні
Клієнт 1: сів у кімнаті очікування
Клієнт 1: стрижеться
Клієнт 2: прийшов до перукарні
Клієнт 2: сів у кімнаті очікування
Клієнт 2: стрижеться
Клієнт 3: прийшов до перукарні
Клієнт 3: сів у кімнаті очікування
Клієнт 0: підстрижений
Клієнт 3: стрижеться
Клієнт 4: прийшов до перукарні
Клієнт 4: сів у кімнаті очікування
Клієнт 5: прийшов до перукарні
Клієнт 5: сів у кімнаті очікування
Клієнт 6: прийшов до перукарні
Клієнт 6: сів у кімнаті очікування
Клієнт 1: підстрижений
Клієнт 4: стрижеться
Клієнт 7: прийшов до перукарні
Клієнт 7: сів у кімнаті очікування
Клієнт 2: підстрижений
Клієнт 5: стрижеться
Клієнт 3: підстрижений
Клієнт 6: стрижеться
Клієнт 8: прийшов до перукарні
Клієнт 8: сів у кімнаті очікування
Клієнт 9: прийшов до перукарні
Клієнт 9: сів у кімнаті очікування
Клієнт 6: підстриженийКлієнт 4: підстрижений
Клієнт 7: стрижеться

Клієнт 8: стрижеться
Клієнт 5: під

13. Побудувати модель завдання 12 для випадку, коли клієнт, якому не дістається місце, може 
або чекати стоячи, або піти непідстриженим.

In [28]:
class FlexibleBarbershop:
    def __init__(self, n_barbers=3, n_seats=5):
        self.barbers = threading.Semaphore(n_barbers)
        self.seats = threading.Semaphore(n_seats)
        
    def client(self, client_id, patient=True):
        print(f"Клієнт {client_id}: прийшов до перукарні")
        
        got_seat = self.seats.acquire(blocking=False)
        
        if got_seat:
            print(f"Клієнт {client_id}: сів у кімнаті очікування")
        elif patient:
            print(f"Клієнт {client_id}: чекає стоячи")
        else:
            print(f"Клієнт {client_id}: йде непідстриженим")
            return
        
        self.barbers.acquire()
        print(f"Клієнт {client_id}: стрижеться")
        
        if got_seat:
            self.seats.release()
        
        time.sleep(random.uniform(0.3, 0.6))
        
        print(f"Клієнт {client_id}: підстрижений")
        self.barbers.release()
    
    def run(self, n_clients=10):
        threads = []
        for i in range(n_clients):
            patient = random.choice([True, False])
            t = threading.Thread(target=self.client, args=(i, patient))
            threads.append(t)
            t.start()
            time.sleep(random.uniform(0.05, 0.15))
        
        for t in threads:
            t.join()

# Запуск
flex_barber = FlexibleBarbershop()
flex_barber.run()



Клієнт 0: прийшов до перукарні
Клієнт 0: сів у кімнаті очікування
Клієнт 0: стрижеться
Клієнт 1: прийшов до перукарні
Клієнт 1: сів у кімнаті очікування
Клієнт 1: стрижеться
Клієнт 2: прийшов до перукарні
Клієнт 2: сів у кімнаті очікування
Клієнт 2: стрижеться
Клієнт 3: прийшов до перукарні
Клієнт 3: сів у кімнаті очікування
Клієнт 4: прийшов до перукарні
Клієнт 4: сів у кімнаті очікування
Клієнт 0: підстрижений
Клієнт 3: стрижеться
Клієнт 5: прийшов до перукарні
Клієнт 5: сів у кімнаті очікування
Клієнт 6: прийшов до перукарні
Клієнт 6: сів у кімнаті очікування
Клієнт 7: прийшов до перукарні
Клієнт 7: сів у кімнаті очікування
Клієнт 1: підстрижений
Клієнт 4: стрижеться
Клієнт 2: підстрижений
Клієнт 5: стрижеться
Клієнт 8: прийшов до перукарні
Клієнт 8: сів у кімнаті очікування
Клієнт 9: прийшов до перукарні
Клієнт 9: сів у кімнаті очікування
Клієнт 3: підстрижений
Клієнт 6: стрижеться
Клієнт 4: підстрижений
Клієнт 7: стрижеться
Клієнт 5: підстрижений
Клієнт 8: стрижеться
Клієнт 6: під

14-17. Побудувати модель безпечного руху на ділянці залізниці, що представлена на схемі. 
Рух односторонній. Дорога складається з перегонів зі світлофорами (8) та датчиками 
зайнятості

In [29]:
class RailwaySystem:
    def __init__(self, tracks_config):
        """
        tracks_config: словник {track_id: [список сусідніх перегонів]}
        """
        self.tracks = {track: threading.Lock() for track in tracks_config.keys()}
        self.config = tracks_config
        
    def train(self, train_id, route):
        print(f"Поїзд {train_id}: починає рух по маршруту {route}")
        
        for i, track in enumerate(route):
            # Чекаємо зелений світлофор (вільний перегін)
            print(f"Поїзд {train_id}: чекає на перегін {track}")
            self.tracks[track].acquire()
            
            print(f"Поїзд {train_id}: рухається по перегону {track}")
            time.sleep(random.uniform(0.2, 0.5))
            
            # Звільняємо попередній перегін
            if i > 0:
                prev_track = route[i-1]
                self.tracks[prev_track].release()
                print(f"Поїзд {train_id}: звільнив перегін {prev_track}")
        
        # Звільняємо останній перегін
        self.tracks[route[-1]].release()
        print(f"Поїзд {train_id}: завершив маршрут")
    
    def run(self, trains_routes):
        threads = []
        for train_id, route in trains_routes.items():
            t = threading.Thread(target=self.train, args=(train_id, route))
            threads.append(t)
            t.start()
            time.sleep(random.uniform(0.1, 0.3))
        
        for t in threads:
            t.join()

# Приклад для схеми A (можна адаптувати під конкретні схеми)
print("Схема A:")
config_a = {
    'T1': [], 'T2': [], 'T3': [], 'T4': [], 'T5': [],
    'T6': [], 'T7': [], 'T8': []
}

railway_a = RailwaySystem(config_a)
trains_a = {
    'Train_1': ['T1', 'T2', 'T3', 'T7'],
    'Train_2': ['T4', 'T5', 'T6'],
}
railway_a.run(trains_a)



Схема A:
Поїзд Train_1: починає рух по маршруту ['T1', 'T2', 'T3', 'T7']
Поїзд Train_1: чекає на перегін T1
Поїзд Train_1: рухається по перегону T1
Поїзд Train_2: починає рух по маршруту ['T4', 'T5', 'T6']
Поїзд Train_2: чекає на перегін T4
Поїзд Train_2: рухається по перегону T4
Поїзд Train_1: чекає на перегін T2
Поїзд Train_1: рухається по перегону T2
Поїзд Train_1: звільнив перегін T1
Поїзд Train_1: чекає на перегін T3
Поїзд Train_1: рухається по перегону T3
Поїзд Train_2: чекає на перегін T5
Поїзд Train_2: рухається по перегону T5
Поїзд Train_1: звільнив перегін T2
Поїзд Train_1: чекає на перегін T7
Поїзд Train_1: рухається по перегону T7
Поїзд Train_2: звільнив перегін T4
Поїзд Train_2: чекає на перегін T6
Поїзд Train_2: рухається по перегону T6
Поїзд Train_2: звільнив перегін T5
Поїзд Train_2: завершив маршрут
Поїзд Train_1: звільнив перегін T3
Поїзд Train_1: завершив маршрут


18. Побудувати модель ядерной реакції, у якій з кожного атому, що стикається з нейтроном, 
виділяється при розпаді вісім нейтронів і створюється новий атом.

In [30]:
class NuclearReaction:
    def __init__(self, initial_atoms=5, neutrons_per_decay=8, max_iterations=3):
        self.atoms = initial_atoms
        self.neutrons = 1  # Початковий нейтрон
        self.neutrons_per_decay = neutrons_per_decay
        self.max_iterations = max_iterations
        self.lock = threading.Lock()
        self.iteration = 0
        
    def atom(self, atom_id):
        print(f"Атом {atom_id}: створено")
        
        while self.iteration < self.max_iterations:
            with self.lock:
                if self.neutrons > 0:
                    self.neutrons -= 1
                    print(f"Атом {atom_id}: зіткнувся з нейтроном")
                else:
                    time.sleep(0.1)
                    continue
            
            # Розпад атому
            print(f"Атом {atom_id}: розпадається")
            time.sleep(random.uniform(0.1, 0.3))
            
            with self.lock:
                self.neutrons += self.neutrons_per_decay
                print(f"Атом {atom_id}: виділив {self.neutrons_per_decay} нейтронів (всього: {self.neutrons})")
                
                # Створюємо новий атом
                new_atom_id = f"{atom_id}_child"
                new_thread = threading.Thread(target=self.atom, args=(new_atom_id,))
                new_thread.start()
                
                self.iteration += 1
            
            break
    
    def run(self):
        print(f"Початок реакції: {self.atoms} атомів, {self.neutrons} нейтрон")
        
        threads = []
        for i in range(self.atoms):
            t = threading.Thread(target=self.atom, args=(i,))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()
        
        print(f"\nКінець реакції. Залишилось нейтронів: {self.neutrons}")

# Запуск
nr = NuclearReaction(initial_atoms=3, neutrons_per_decay=8, max_iterations=2)
nr.run()

Початок реакції: 3 атомів, 1 нейтрон
Атом 0: створено
Атом 0: зіткнувся з нейтроном
Атом 0: розпадається
Атом 1: створено
Атом 2: створено
Атом 0: виділив 8 нейтронів (всього: 8)
Атом 0_child: створено
Атом 1: зіткнувся з нейтроном
Атом 1: розпадається
Атом 2: зіткнувся з нейтроном
Атом 2: розпадається
Атом 0_child: зіткнувся з нейтроном
Атом 0_child: розпадається
Атом 2: виділив 8 нейтронів (всього: 13)
Атом 2_child: створено
Атом 2_child: зіткнувся з нейтроном
Атом 2_child: розпадається
Атом 0_child: виділив 8 нейтронів (всього: 20)
Атом 0_child_child: створено
Атом 2_child: виділив 8 нейтронів (всього: 28)
Атом 2_child_child: створено
Атом 1: виділив 8 нейтронів (всього: 36)
Атом 1_child: створено

Кінець реакції. Залишилось нейтронів: 36
